# 1. 환결설정

### 라이브러리 로드

In [4]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments, StoppingCriteria, StoppingCriteriaList
from trl import SFTTrainer
import os
import random

from datasets import Dataset, load_dataset
import pandas as pd
import numpy as np
import re

import torch
import wandb
from tqdm.notebook import tqdm
from transformers import TextStreamer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.5.1+cu124)
    Python  3.11.11 (you have 3.11.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


### CFG 파라미터 설정

In [ ]:
CFG = {
    'MODEL_NAME': "yanolja/EEVE-Korean-Instruct-10.8B-v1.0",
    'LEARNING_RATE': 3e-4,
    # 'STEPS': 8361,
    'SEED': 42
}

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(CFG['SEED']) # Seed 고정

### 모델 로드

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name= CFG['MODEL_NAME'],
    max_seq_length=4096,
    dtype=None,
    load_in_4bit=True,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=CFG["SEED"],
    use_rslora=False,
    loftq_config=None,
)



==((====))==  Unsloth 2025.2.15: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = None. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Unsloth 2025.2.15 patched 48 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


# 2. 데이터 전처리

### 데이터 로드

In [8]:
train = pd.read_csv('/content/drive/MyDrive/dacon/open/train.csv', encoding = 'utf-8-sig')
test = pd.read_csv('/content/drive/MyDrive/dacon/open/test.csv', encoding = 'utf-8-sig')

train = train.drop(columns="ID")


### 데이터 증강

In [ ]:

# 초성 리스트. 00 ~ 18
CHOSUNG_LIST = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
# 중성 리스트. 00 ~ 20
JUNGSUNG_LIST = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
DOUBLE_JONGSUNG_LIST = ['ㄲ', 'ㄳ', 'ㄵ', 'ㄶ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ',  'ㄿ', 'ㅀ', 'ㅄ',  'ㅆ']

#자모 비슷한 발음 딕셔너리 정리
#초성 비슷한 발음 딕셔너리
SIMILAR_CHOSUNG_LIST = {
    'ㄱ' : ['ㄲ', 'ㅋ'],
    'ㄲ' : ['ㄱ', 'ㅋ'],
    'ㄷ' : ['ㄸ', 'ㅌ'],
    'ㄸ' : ['ㄷ', 'ㅌ'],
    'ㅂ' : ['ㅃ', 'ㅍ'],
    'ㅃ' : ['ㅂ', 'ㅍ'],
    'ㅅ' : ['ㅆ'],
    'ㅆ' : ['ㅅ'],
    'ㅈ' : ['ㅉ', 'ㅊ'],
    'ㅉ' : ['ㅈ', 'ㅊ'],
    'ㅊ' : ['ㅈ', 'ㅉ'],
    'ㅋ' : ['ㄲ'],
    'ㅌ' : ['ㄸ'],
    'ㅍ' : ['ㅃ']
                }

#중성 비슷한 발음 딕셔너리
SIMILAR_JUNGSUNG_LIST = {
    'ㅏ' : ['ㅑ', 'ㅘ'],
    'ㅐ' : ['ㅒ', 'ㅔ', 'ㅖ', 'ㅙ', 'ㅚ', 'ㅞ'],
    'ㅑ' : ['ㅏ'],
    'ㅒ' : ['ㅐ', 'ㅔ', 'ㅖ', 'ㅙ', 'ㅚ', 'ㅞ'],
    'ㅓ' : ['ㅕ', 'ㅗ'],
    'ㅔ' : ['ㅒ', 'ㅐ', 'ㅖ', 'ㅙ', 'ㅚ', 'ㅞ'],
    'ㅕ' : ['ㅓ', 'ㅛ'],
    'ㅗ' : ['ㅓ', 'ㅛ', 'ㅝ'],
    'ㅘ' : ['ㅙ', 'ㅗ'],
    'ㅙ' : ['ㅚ', 'ㅐ', 'ㅞ', 'ㅔ', 'ㅒ', 'ㅖ'],
    'ㅚ' : ['ㅙ', 'ㅐ', 'ㅞ', 'ㅔ', 'ㅒ', 'ㅖ'],
    'ㅛ' : ['ㅗ', 'ㅕ'],
    'ㅜ' : ['ㅝ', 'ㅠ', 'ㅡ'],
    'ㅝ' : ['ㅞ', 'ㅗ', 'ㅓ', 'ㅚ', 'ㅡ'],
    'ㅞ' : ['ㅙ', 'ㅐ', 'ㅚ', 'ㅔ', 'ㅒ', 'ㅖ'],
    'ㅟ' : ['ㅣ', 'ㅚ', 'ㅢ'],
    'ㅠ' : ['ㅜ', 'ㅡ'],
    'ㅡ' : ['ㅜ', 'ㅢ'],
    'ㅢ' : ['ㅣ', 'ㅡ'],
    'ㅣ' : ['ㅢ', 'ㅟ']
}

#종성 비슷한 발음 딕셔너리
SIMILAR_JONGSUNG_LIST = {
    'ㄱ' : ['ㄲ', 'ㅋ', 'ㄳ'],
    'ㄲ' : ['ㄱ', 'ㅋ', 'ㄳ'],
    'ㄳ' : ['ㄱ', 'ㅅ'],
    'ㄴ' : ['ㄵ'],
    'ㄵ' : ['ㄴ'],
    'ㄶ' : ['ㄴ'],
    'ㄷ' : ['ㅅ', 'ㅆ', 'ㅈ', 'ㅌ'],
    'ㄹ' : ['ㄼ'],
    'ㄺ' : ['ㄹ', 'ㄺ', 'ㄻ'],
    'ㄻ' : ['ㄹ', 'ㄺ'],
    'ㄼ' : ['ㄹ', 'ㄺ', 'ㄻ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ'],
    'ㄽ' : ['ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄾ', 'ㄿ', 'ㅀ'],
    'ㄾ' : ['ㄹ', 'ㄺ', 'ㄻ', 'ㄽ', 'ㄼ', 'ㄿ', 'ㅀ'],
    'ㄿ' : ['ㄹ', 'ㄺ', 'ㄻ', 'ㄽ', 'ㄼ', 'ㄾ', 'ㅀ'],
    'ㅀ' : ['ㄹ', 'ㄺ', 'ㄻ', 'ㄽ', 'ㄼ', 'ㄾ', 'ㄿ'],
    'ㅁ' : ['ㄻ', 'ㄼ'],
    'ㅂ' : ['ㅍ', 'ㅄ'],
    'ㅄ' : ['ㅂ','ㅍ'],
    'ㅅ' : ['ㅆ', 'ㅈ', 'ㅊ', 'ㄷ', 'ㅌ'],
    'ㅆ' : ['ㅅ', 'ㅈ', 'ㅊ', 'ㄷ', 'ㅌ'],
    'ㅇ' : ['ㅎ'],
    'ㅈ' : ['ㅆ', 'ㅅ', 'ㅊ', 'ㄷ', 'ㅌ'],
    'ㅊ' : ['ㅆ', 'ㅈ', 'ㅅ', 'ㄷ', 'ㅌ'],
    'ㅋ' : ['ㄲ', 'ㄱ', 'ㄳ'],
    'ㅌ' : ['ㅆ', 'ㅈ', 'ㅅ', 'ㄷ', 'ㅊ'],
    'ㅍ' : ['ㅂ', 'ㅄ'],
    'ㅎ' : ['ㅎ'],
}

def split_korean(korean_word):
    r_lst = []
    for w in list(korean_word.strip()):
        ## 영어인 경우 구분해서 작성함.
        if '가'<=w<='힣':
            ## 588개 마다 초성이 바뀜.
            ch1 = (ord(w) - ord('가'))//588
            ## 중성은 총 28가지 종류
            ch2 = ((ord(w) - ord('가')) - (588*ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588*ch1) - 28*ch2
            r_lst.append([CHOSUNG_LIST[ch1], JUNGSUNG_LIST[ch2], JONGSUNG_LIST[ch3]])
        else:
            r_lst.append([w])
    return r_lst


def transform_korean(korean_word, seed=42, chosung_prob=0.5, jungsung_prob=0.5, jongsung_prob=0.5, jongsung_add_prob=0.5):
    # 랜덤 시드 설정 (일관된 결과를 위해)
    if seed is not None:
        random.seed(seed)

    #소리나는 대로 연음법칙 적용 기능 구현


    #뒤에 오는 자음을 받침으로 중복 기능 구현

    # 변환 확률을 설정하여 변환을 수행
    def get_similar_item(d, item, prob):
        if item in d and random.random() < prob:
            return random.choice(d[item])
        return item

    r_lst = []
    for w in list(korean_word.strip()):
        if '가' <= w <= '힣':
            # 한글을 초성, 중성, 종성으로 분해
            ch1 = (ord(w) - ord('가')) // 588
            ch2 = ((ord(w) - ord('가')) - (588 * ch1)) // 28
            ch3 = (ord(w) - ord('가')) - (588 * ch1) - 28 * ch2

            chosung = get_similar_item(SIMILAR_CHOSUNG_LIST, CHOSUNG_LIST[ch1], chosung_prob)
            jungsung = get_similar_item(SIMILAR_JUNGSUNG_LIST, JUNGSUNG_LIST[ch2], jungsung_prob)

            # 종성이 없으면 확률에 따라 종성 추가
            if JONGSUNG_LIST[ch3] == ' ' and random.random() < jongsung_add_prob:
                # 종성이 없을 경우 무작위로 종성 추가 (기본값: 'ㄱ' 추가)
                jongsung = random.choice(JONGSUNG_LIST[1:])  # 공백이 아닌 종성 중 하나를 선택
            else:
                jongsung = get_similar_item(SIMILAR_JONGSUNG_LIST, JONGSUNG_LIST[ch3], jongsung_prob)

            # 변환된 한글을 재조합하여 추가
            new_char = chr(ord('가') + CHOSUNG_LIST.index(chosung) * 588 + JUNGSUNG_LIST.index(jungsung) * 28 + JONGSUNG_LIST.index(jongsung))
            r_lst.append(new_char)
        else:
            # 한글이 아닌 경우 그대로 추가
            r_lst.append(w)

    # 변환된 리스트를 문자열로 반환
    return ''.join(r_lst)


# print("정답:", train.loc[0, 'output'])
# print("입력:", train.loc[0, 'input'], '\n')

# for i in range(5):
#     transformed_text = transform_korean(train.loc[0, 'output'], seed=i)
#     print("증강:", transformed_text)



seed_lst = [42]
original_train = train.copy()

for seed in seed_lst:
    for i in range(len(original_train)):
        transformed_text = transform_korean(original_train.loc[i, 'output'], seed=seed)
        new_row = pd.DataFrame([[transformed_text, train.loc[i, 'output']]], columns=['input', 'output'])
        train = pd.concat([train, new_row], ignore_index=True)



In [ ]:
validation_clean = "(가) 가는 길이지만 (나) 나를 생각해요 (다) 다른 사람에게 벌써 마음을 주지 마요 (라) 라일락 꽃잎들을 (마) 마음껏 밟고 가요 (바) 바라는 건 딱 한 가지 끝까지 나를 생각해요 (사) 사라져 버릴까봐서 겁나요 (아자차카타파) 파란 하늘 보며 가슴을 펴고 하하 웃지 말아요 (가) 가랑비라도 (나) 나리는 날엔 (다) 다시 한번 나를 떠올려줘요 (가) 가벼운 발걸음 (나) 나는 싫어요 (다) 다 잊었다 제발 생각 말아요 (가) 가끔 생각이 (나) 나면 (다) 다 잊은 뒤에 (라) 라도 (마) 마음 터 놓고 만나기를 (바) 바라는 건 욕심이겠죠 사르르 녹아버릴까봐 겁나요 (아자차카타파) 파르르 떨리는 입술을 물고 하염없이 울어요 (가) 가랑비라도 (나) 나리는 날엔 (다) 다시 한번 나를 떠올려줘요 (가) 가벼운 발걸음 (나) 나는 싫어요 (다) 다 잊었다 제발 생각 말아요 (가) 가로등불이 (나) 나를 만지면 (다) 다시 너랑 둘이 걷고 싶겠지 (가) 가끔이라도 (나) 나와 단둘이 (다) 다정했던 우리 밤과 낮을 생각해 줘요"
validation_dirty = transform_korean(validation_clean)
validation_dirty

'(괁) 가늱 낄잃짚뫉 (놡) 놔를 샣각해오 (다) 탌릔 샤뢂에걔 뽋셦 마읢읇 줘쮜 마오 (랑) 뢇윌락 콫윞들읣 (마) 맊욻껏 팙거 가엽 (뽮) 뺚랄늱 건 탂 핝 까치 긪갚짒 나를 쐥깤훼요 (솪) 사롸저 뻔릴꺗봋쏱 곂낤요 (얎자촤꺄따빢) 빫란 화눏 보멄 가숨읠 펺고 화햪 웄쥫 맓앍요 (카) 꽑뢓삋랷도 (낢) 나뤼늰 날왼 (똻) 다시 홙볁 놔륿 똥올려즚옾 (카) 가벼욵 밟결읢 (냕) 낦늱 싏억욡 (닼) 다 윘옸다 제봛 쎟곾 먈야요 (갸) 카큶 솋각읯 (낯) 나멵 (탕) 탌 잊읁 뒤웨 (랴) 뢇돔 (맏) 뫘읆 텨 넣걸 먅놬귉룳 (퐈) 봑띾는 곥 욬쓂의굈쪄 솷릤루 넊야폴륄가봐 껍낞요 (아차찫카탸파) 팎릅르 턻륒눉 윕슓읇 물꼬 화엄옵익 욻여여 (캆) 까랑퓌롸도 (놦) 낣리늱 놜엕 (똬) 돥쓏 햔벉 나룰 터얼렪줲엱 (카) 깢뿆윤 봘골읣 (낥) 나늕 씷어욮 (따) 다 잊얻탊 채봘 쌯갺 말아욮 (캊) 과롤틓븗의 (놜) 낧릘 만지멵 (타) 타씌 넏랑 퉐의 겯거 십켗취 (과) 깦끎위럃떩 (나) 났왔 딴둛잏 (탋) 댜쩧햊던 웢맄 팖쾞 놪을 생괔핿 쭤오'

In [ ]:
train

### 데이터 변환

In [9]:
def llama_formatting(examples):
  inputs = []
  outputs =[]
  for inp, out in zip(examples['input'], examples['output']):
    inputs.append(inp)
    outputs.append(out)
  return {"input" : inputs, "output": outputs}

llama_dataset = llama_formatting(train) #그냥 딕셔너리임.
llama_dataset_prompt_map = Dataset.from_dict(llama_dataset)


In [ ]:
#11263.0
# display(llama_dataset_prompt_map[0])
# display(llama_dataset_prompt_map[11263])

### EOS 토큰 추출

In [10]:
EOS_TOKEN = tokenizer.eos_token

In [11]:
alpaca_prompt = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, output in zip(instructions, outputs):
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

dataset = llama_dataset_prompt_map

dataset = dataset.map(
    formatting_prompts_func,
    batched=True,
)

Map:   0%|          | 0/11263 [00:00<?, ? examples/s]

### 훈련/검증 데이터셋 분할

In [ ]:
split_dataset = dataset.train_test_split(test_size=0.01, seed=42)
split_dataset
train_dataset = split_dataset["train"]
valid_dataset = split_dataset["test"]

# 3. 훈련

### SFTTrainer 설정

In [ ]:
tokenizer.padding_side = "right"  # 토크나이저의 패딩을 오른쪽으로 설정합니다.

# SFTTrainer를 사용하여 모델 학습 설정
trainer = SFTTrainer(
    model=model,  # 학습할 모델
    tokenizer=tokenizer,  # 토크나이저
    train_dataset=train_dataset,  # 학습 데이터셋
    eval_dataset=valid_dataset,
    dataset_text_field="text",  # 데이터셋에서 텍스트 필드의 이름
    max_seq_length=4096,  # 최대 시퀀스 길이
    dataset_num_proc=8,  # 데이터 처리에 사용할 프로세스 수
    packing=False,  # 짧은 시퀀스에 대한 학습 속도를 5배 빠르게 할 수 있음
    args=TrainingArguments(
        per_device_train_batch_size=2,  # 각 디바이스당 훈련 배치 크기
        gradient_accumulation_steps=4,  # 그래디언트 누적 단계
        warmup_steps=5,  # 웜업 스텝 수
        num_train_epochs=3,  # 훈련 에폭 수
        # max_steps=CFG['STEPS'],  # 최대 스텝 수
        do_eval=True,
        evaluation_strategy="steps",
        eval_steps=500,
        logging_steps=1,  # logging 스텝 수
        learning_rate=CFG['LEARNING_RATE'],  # 학습률
        fp16=not torch.cuda.is_bf16_supported(),  # fp16 사용 여부, bf16이 지원되지 않는 경우에만 사용
        bf16=torch.cuda.is_bf16_supported(),  # bf16 사용 여부, bf16이 지원되는 경우에만 사용
        optim="adamw_8bit",  # 최적화 알고리즘
        weight_decay=0.01,  # 가중치 감소
        lr_scheduler_type="cosine",  # 학습률 스케줄러 유형
        seed=CFG['SEED'],  # 랜덤 시드
        output_dir="model",  # 출력 디렉토리
    ),
)

### Wandb

In [ ]:
wandb.init(project="[DACON] Decryption", name= CFG["MODEL_NAME"])

### 훈련 실행

In [ ]:
trainer_stats = trainer.train()

### 모델 저장

In [ ]:
save_dir = f"./model/{CFG['MODEL_NAME']}"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

('./model/yanolja/EEVE-Korean-Instruct-10.8B-v1.0/tokenizer_config.json',
 './model/yanolja/EEVE-Korean-Instruct-10.8B-v1.0/special_tokens_map.json',
 './model/yanolja/EEVE-Korean-Instruct-10.8B-v1.0/tokenizer.json')

# 4. 추론

### 정지 토큰 설정

In [12]:


class StopOnToken(StoppingCriteria):
    def __init__(self, stop_token_id):
        self.stop_token_id = stop_token_id  # 정지 토큰 ID를 초기화합니다.

    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )  # 입력된 ID 중 정지 토큰 ID가 있으면 정지합니다.


# end_token을 설정
stop_token = EOS_TOKEN
stop_token_id = tokenizer.encode(stop_token, add_special_tokens=False)[
    0
]  # end_token의 ID를 인코딩합니다.

# Stopping criteria 설정
stopping_criteria = StoppingCriteriaList(
    [StopOnToken(stop_token_id)]
)  # 정지 조건을 설정합니다.

### 추론 실행

In [ ]:
test_result = []
FastLanguageModel.for_inference(model) # for_inference는 학습을 비활성화시켜서 추론 속도를 2배 빠르게 만듦
number = 0
for test_i in tqdm(test["input"]):

  # 모델 테스트 모듈

  inputs = tokenizer(
  [
      alpaca_prompt.format(
          f"""{test_i}""", # 질문
          "",                                              # 생성된 답변칸 비어두기
      )
  ], return_tensors = "pt").to("cuda") # PyTorch 텐서, GPU사용

  outputs = model.generate(**inputs,            # 이전 입력값
                           max_new_tokens = 1800,
                           use_cache=True ,
                           stopping_criteria=stopping_criteria)    # 캐싱을 사용하여 이전 결과 저장

  t =tokenizer.batch_decode(outputs,skip_special_tokens=True)[0]
  match = re.search(r'Response:\n\s*(.*)', t, re.DOTALL)
  if match:
    output_text = match.group(1).strip()
  else:
    #print(f"[1]:{t}")
    output_text = test_i
  print(f"{number} : {output_text}")
  test_result.append(output_text)
  number += 1

  0%|          | 0/1689 [00:00<?, ?it/s]

KeyboardInterrupt: 

### save result

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8-sig')
submission['output'] = test_result
submission.to_csv('./submission/yanolja_EEVE-Korean-Instruct-10.8B-v1_AUG_2x_epohc_3.csv', index = False, encoding = 'utf-8-sig')
submission

,ID,output
0,TEST_0000,너무너무 만족스러운 호텔이에요. 부산에 오면 꼭 추천하고 싶은 곳이에요. 최고입니다...
1,TEST_0001,"프론트가 없고, 조식도 없으며, 일반 입주민들이 사이사에 있어 호텔처럼 관리가 잘 ..."
2,TEST_0002,진짜 불친절해요. 살면서 머물렀던 호텔 중에 최악이었습니다. 직원인지 사장인지 체크...
3,TEST_0003,뷰 맛집~~ 그런데 방음이 미흡하네요. 층간 소음과 발코니가 윗층층이 아니라서 밤에...
4,TEST_0004,방 상태는 진짜 폐허 지저인데 전망은 좋아요. 보일러가 아주 잔잔하게 돌아가서 추웠...
...,...,...
1684,TEST_1684,"일반실 5만원이고, 프리미엄 6만원짜리는 스타일러스, 안마기도 있고, PC도 있지만..."
1685,TEST_1685,"일단 방은 이쁘고, 배달 음식은 끔찍어서 (외인지는 의문..) 룸서비스랑 조식으로 ..."
1686,TEST_1686,"웬만하면 다시 안 올 예정입니다. 겨울 외풍이 있습니다. 실골. 이건 뭐, 관리할 ..."
1687,TEST_1687,도착 후 크게 기대하지 않았는데 직원분들이 친절하시고 2박 머무는 동안 매일 객실도...
